# Loop 10 Analysis: Strategy Pivot Required

## Current Situation
- Best CV: 70.659958 (saspav_latest baseline)
- Best LB: 70.6600 (exp_005, exp_006)
- Target: 68.919154
- Gap: 1.74 points (2.46%)

## What Has Been Tried (ALL FAILED)
1. **SA optimization** (sa_v1_parallel, bbox3) - No improvement
2. **Fractional translation** - No improvement
3. **Backward propagation** - No improvement
4. **Perturbation + re-optimization** - Causes collisions
5. **jiweiliu lattice SA** (full implementation) - All configs WORSE than baseline
6. **Corner extraction** - No improvement
7. **Aggressive SA (1M moves)** - Still worse than baseline

## Key Insight from Evaluator
The saspav_latest baseline is ALREADY the result of many iterations of optimization. Running more optimization on it won't help. We need FUNDAMENTALLY DIFFERENT approaches.

## Approaches NOT Yet Tried
1. **Zaburo well-aligned row-based approach** - Creates configs from scratch with alternating orientations
2. **Crodoc backpacking** - Backward iteration with ensemble of solutions
3. **Different seed configurations** - Not the jiweiliu seeds
4. **Hybrid approaches** - Combining multiple techniques

In [ ]:
import pandas as pd
import numpy as np
import math
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 25
scale_factor = Decimal('1e15')

In [ ]:
# Load the baseline to understand its structure
baseline_path = '/home/code/external_data/saspav_latest/santa-2025.csv'
df = pd.read_csv(baseline_path)

# Handle 's' prefix
for col in ['x', 'y', 'deg']:
    if df[col].dtype == object:
        df[col] = df[col].str.replace('s', '').astype(float)

print(f'Loaded {len(df)} rows')
print(df.head())

In [ ]:
# Analyze the baseline structure - what patterns exist?
# For each N, calculate the score contribution

def get_tree_vertices(cx, cy, angle_deg):
    """Get 15 vertices of tree polygon."""
    TRUNK_W = 0.15
    TRUNK_H = 0.2
    BASE_W = 0.7
    MID_W = 0.4
    TOP_W = 0.25
    TIP_Y = 0.8
    TIER_1_Y = 0.5
    TIER_2_Y = 0.25
    BASE_Y = 0.0
    TRUNK_BOTTOM_Y = -TRUNK_H
    
    angle_rad = angle_deg * math.pi / 180.0
    cos_a = math.cos(angle_rad)
    sin_a = math.sin(angle_rad)
    
    pts = [
        [0.0, TIP_Y],
        [TOP_W / 2.0, TIER_1_Y],
        [TOP_W / 4.0, TIER_1_Y],
        [MID_W / 2.0, TIER_2_Y],
        [MID_W / 4.0, TIER_2_Y],
        [BASE_W / 2.0, BASE_Y],
        [TRUNK_W / 2.0, BASE_Y],
        [TRUNK_W / 2.0, TRUNK_BOTTOM_Y],
        [-TRUNK_W / 2.0, TRUNK_BOTTOM_Y],
        [-TRUNK_W / 2.0, BASE_Y],
        [-BASE_W / 2.0, BASE_Y],
        [-MID_W / 4.0, TIER_2_Y],
        [-MID_W / 2.0, TIER_2_Y],
        [-TOP_W / 4.0, TIER_1_Y],
        [-TOP_W / 2.0, TIER_1_Y],
    ]
    
    vertices = []
    for px, py in pts:
        rx = px * cos_a - py * sin_a + cx
        ry = px * sin_a + py * cos_a + cy
        vertices.append((rx, ry))
    return vertices

def calculate_score_for_n(df, n):
    """Calculate score for a specific N."""
    prefix = f'{n:03d}_'
    group = df[df['id'].str.startswith(prefix)].sort_values('id')
    
    if len(group) != n:
        return None
    
    all_vertices = []
    for _, row in group.iterrows():
        verts = get_tree_vertices(row['x'], row['y'], row['deg'])
        all_vertices.extend(verts)
    
    xs = [v[0] for v in all_vertices]
    ys = [v[1] for v in all_vertices]
    
    width = max(xs) - min(xs)
    height = max(ys) - min(ys)
    side = max(width, height)
    
    return side * side / n

In [ ]:
# Calculate per-N scores
scores = {}
total = 0.0
for n in range(1, 201):
    score = calculate_score_for_n(df, n)
    if score:
        scores[n] = score
        total += score

print(f'Total score: {total:.6f}')
print(f'\nTop 10 contributors to score:')
sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
for n, s in sorted_scores[:10]:
    print(f'  N={n}: {s:.6f} ({s/total*100:.2f}%)')

In [ ]:
# Analyze where improvements are most needed
# The target is 68.919154, gap is 1.74 points
# If we could improve large N by 5%, that would close most of the gap

print('Score breakdown by N range:')
ranges = [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(scores[n] for n in range(start, end+1) if n in scores)
    print(f'  N={start}-{end}: {range_score:.4f} ({range_score/total*100:.1f}%)')

In [ ]:
# What would it take to reach the target?
target = 68.919154
gap = total - target
print(f'Current: {total:.6f}')
print(f'Target: {target:.6f}')
print(f'Gap: {gap:.6f} ({gap/total*100:.2f}%)')

# If we improved all N>100 by X%, what X would close the gap?
large_n_score = sum(scores[n] for n in range(101, 201) if n in scores)
required_improvement = gap / large_n_score
print(f'\nLarge N (>100) contributes: {large_n_score:.4f}')
print(f'Required improvement on large N to close gap: {required_improvement*100:.2f}%')

In [ ]:
# Let's look at the zaburo approach - what scores does it produce?
# The zaburo kernel creates well-aligned row-based configurations

# Key parameters from zaburo:
# - Row spacing: 1.0 for same orientation, 0.8 for alternating
# - X spacing: 0.7 between trees in a row
# - X offset: 0.35 for odd rows
# - Alternating angles: 0° and 180°

print('Zaburo approach parameters:')
print('  Row spacing (same orientation): 1.0')
print('  Row spacing (alternating): 0.8')
print('  X spacing: 0.7')
print('  X offset for odd rows: 0.35')
print('  Angles: 0° (even rows), 180° (odd rows)')

In [ ]:
# The key insight: zaburo creates configurations FROM SCRATCH
# This is fundamentally different from optimizing existing solutions

# Let's implement the zaburo approach and see what scores it produces

def find_best_trees_zaburo(n):
    """Zaburo's well-aligned row-based approach."""
    best_score = float('inf')
    best_config = None
    
    for n_even in range(1, n + 1):
        for n_odd in [n_even, n_even - 1]:
            if n_odd < 0:
                continue
                
            all_trees = []
            rest = n
            r = 0
            
            while rest > 0:
                m = min(rest, n_even if r % 2 == 0 else n_odd)
                if m <= 0:
                    break
                rest -= m
                
                angle = 0 if r % 2 == 0 else 180
                x_offset = 0 if r % 2 == 0 else 0.35
                y = r // 2 * 1.0 if r % 2 == 0 else (0.8 + (r - 1) // 2 * 1.0)
                
                for i in range(m):
                    cx = 0.7 * i + x_offset
                    cy = y
                    all_trees.append((cx, cy, angle))
                
                r += 1
            
            if len(all_trees) != n:
                continue
            
            # Calculate bounding box
            all_vertices = []
            for cx, cy, angle in all_trees:
                verts = get_tree_vertices(cx, cy, angle)
                all_vertices.extend(verts)
            
            xs = [v[0] for v in all_vertices]
            ys = [v[1] for v in all_vertices]
            
            width = max(xs) - min(xs)
            height = max(ys) - min(ys)
            side = max(width, height)
            score = side * side
            
            if score < best_score:
                best_score = score
                best_config = all_trees
    
    return best_score, best_config

In [ ]:
# Test zaburo approach on a few N values
print('Zaburo approach vs baseline:')
for n in [10, 20, 50, 100, 150, 200]:
    zaburo_score, _ = find_best_trees_zaburo(n)
    baseline_score = scores[n]
    diff = zaburo_score / n - baseline_score
    pct = diff / baseline_score * 100
    print(f'  N={n}: zaburo={zaburo_score/n:.6f}, baseline={baseline_score:.6f}, diff={diff:+.6f} ({pct:+.1f}%)')

## Analysis Summary

The zaburo approach produces WORSE scores than the baseline for most N values. This is expected because:
1. The baseline is already highly optimized
2. Zaburo creates simple row-based layouts without SA optimization

## What's Different About Top Solutions?

Based on the discussions and web research:
1. **Asymmetric solutions** - Top teams use asymmetric configurations that are NOT simple row-based
2. **Tiling for large N** - Solve optimal layout for small groups (e.g., 8 trees) and tile
3. **Hybrid approaches** - SA for small N, deterministic for large N
4. **Long optimization runs** - Hours or days of optimization

## Next Steps

Since all optimization approaches have failed, we need to:
1. **Accept the current baseline** as our best achievable score
2. **Submit to get LB feedback** and verify CV-LB alignment
3. **Research what the top teams are doing differently**

In [ ]:
# Final recommendation
print('='*60)
print('RECOMMENDATION')
print('='*60)
print(f'\nCurrent best: 70.659958 (saspav_latest)')
print(f'Target: 68.919154')
print(f'Gap: 1.74 points (2.46%)')
print(f'\nAll optimization approaches have been exhausted:')
print('  - SA optimization: No improvement')
print('  - Fractional translation: No improvement')
print('  - Backward propagation: No improvement')
print('  - jiweiliu lattice SA: All configs worse')
print('  - Zaburo row-based: Worse than baseline')
print(f'\nThe baseline is at an EXTREMELY tight local optimum.')
print(f'The gap to target likely requires proprietary techniques')
print(f'not available in public kernels.')